In [327]:
import pandas as pd
import json
import networkx as nx
from networkx_query import search_nodes, search_edges
import numpy as np
import math

In [328]:
with open('/Users/theojolliffe/Documents/Wayback BBC/topicScoresOrder.json', 'r') as fp:
    topicScoresOrder = json.load(fp)
topicScoresOrder = {i: {i[0]: 1/(1+i[1]/10) for i in topicScoresOrder[i]} for i in topicScoresOrder}

In [329]:
options = pd.read_csv('./csv/lists/places_2020.csv')
areaType = pd.read_csv('/Users/theojolliffe/Documents/Census Data/censusAreaLookup.csv')
typeLookup = {}
for i in areaType.index:
    typeLookup[areaType.iloc[i]['Name']]=areaType.iloc[i]['Group name']

areas = []
for i in options["code"]:
    try:
        areas.append(json.load(open(f'/Users/theojolliffe/Documents/pipeline_v3/final/json/place/{i}.json', 'rb')))
    except FileNotFoundError:
#         print(i)
        pass
    
for area in areas:
    area['data']['agemed']['value']['change']['all'] = area['data']['agemed']['value']['2011']['all']-area['data']['agemed']['value']['2001']['all']
    area['data']['density']['value']['change']['all'] = area['data']['density']['value']['2011']['all']-area['data']['density']['value']['2001']['all']

In [330]:
# Seperate the areas by area type
regions = []
for i in areas:
    if i['type']=='rgn':
        regions.append(i)
        
lads = []
for i in areas:
    if i['type']=='lad':
        lads.append(i)
        
countries = []
for i in areas:
    if (i['type']=='ew')|(i['type']=='ctry'):
        countries.append(i)

In [331]:
nameChanges = {
    "Herefordshire, County of": "Herefordshire",
    "Bristol, City of": "Bristol",
    "Isle of Anglesey": "the Isle of Anglesey"
}

In [332]:
simiDF = pd.read_csv('/Users/theojolliffe/Documents/correspondinglocalauthoritiesv3r.csv')

In [333]:
noOfLads = {"England": 307, "Wales": 22}
areaClassDF = pd.read_csv('/Users/theojolliffe/Documents/area-reports/src/data/censusAreaLookup.csv')
areaClassDF.at[75, 'Code'] = "E06000058"
areaClassLU = {
    "1a1r": "affluent suburban area", "1b1r": "affluent rural area", 
    "1b2r": "growing rural area", "2a1r": "city", 
    "2b1r": "area", "3a2r": "remote area", 
    "3a1r": "agricultural area", "3b1r": "coastal area",
    "3b2r": "seaside  area",
    "4a1r": "urban area", "5a1r": "cosmopolitan area",
    "6a2r": "industrial area", "6a3r": "service economy area",
    "6a1r": "industrial area", "7a1r": "rural area",
    "7c2r": "affluent area", "7c1r": "affluent area",
    "8a1r": "multi-cultural area", "8a2r": "urban area",
    "8b1r": "suburban area", "8b2r": "suburban area",
}
areaClassSuperLU = {
    "1r": "affluent area",
    "2r": "urban area",
    "3r": "rural area",
    "4r": "multi-cultural area",
    "5r": "cosmopolitan area",
    "6r": "industrial area",
    "7r": "suburban area",
    "8r": "built up area",
}
gssLookup = {"E09": "London borough", "E08": "metropolitan district", "E07": "district", "E06": "unitary authority", "W06": "Welsh district"}
gssLookupShort = {"E09": "borough", "E08": "district", "E07": "district", "E06": "unitary authority", "W06": "district"}


In [334]:
bbcReg = pd.read_csv('/Users/theojolliffe/Downloads/bbc_regions.csv')
bbcRegLU = {}
for i in bbcReg.index:
    if bbcReg['areaCode'].iloc[i] in [lad['code'] for lad in lads]:
        bbcRegLU[bbcReg['areaCode'].iloc[i]] = bbcReg['bbc_online'].iloc[i]
bbcRegLU['E06000053'] = 'cornwall'
bbcRegLU['E06000060'] = 'beds_bucks_and_herts'
bbcRegLU['E09000001'] = 'london'
bbcRegLU['E07000150'] = 'northamptonshire'

regionsBBC = ['cumbria', 'lancashire', 'merseyside', 'manchester', 'tees', 'tyne_and_wear', 'Humberside', 'york_and_north_yorkshire', 'leeds_and_west_yorkshire', 'lincolnshire', 'south_yorkshire', 'birmingham_and_black_country', 'coventry_and_warwickshire', 'hereford_and_worcester', 'shropshire', 'stoke_and_staffordshire', 'derbyshire', 'leicester', 'northamptonshire', 'nottingham', 'bristol', 'cornwall', 'devon', 'gloucestershire', 'somerset', 'wiltshire', 'beds_bucks_and_herts', 'cambridgeshire', 'essex', 'norfolk', 'suffolk', 'berkshire', 'dorset', 'hampshire', 'oxford', 'kent', 'london', 'surrey', 'sussex']
bbcNames = {
     'derby': 'derbyshire', 
     'hereford and worcester': 'hereford_and_worcester',
     'tyne and wear': 'tyne_and_wear',
     'northampton': 'northamptonshire',
     'liverpool': 'merseyside',
     'coventry and warwickshire': 'coventry_and_warwickshire',
     'humberside': 'Humberside',
     'sheffield and south yorkshire': 'south_yorkshire',
     'hampshire & isle of wight': 'hampshire',
     'stoke and staffordshire': 'stoke_and_staffordshire',
     'york & north yorkshire': 'york_and_north_yorkshire',
     'birmingham and black country': 'birmingham_and_black_country',
     'beds, bucks and herts': 'beds_bucks_and_herts',
     'leeds and west yorkshire': 'leeds_and_west_yorkshire'
}
for i in bbcRegLU.keys():
    bbcRegLU[i] = bbcRegLU[i].lower()
    if bbcRegLU[i] not in regionsBBC:
        try:
            bbcRegLU[i] = bbcNames[bbcRegLU[i]]
        except:
            pass

In [335]:
tfidf = pd.read_csv('/Users/theojolliffe/Documents/Wayback BBC/BBCRegionalTFIDF.csv')
tfidf = tfidf.set_index('Unnamed: 0')
for i in tfidf.columns:
    for j in tfidf[i].index:
        tfidf[i].loc[j] = 1+100*tfidf[i].loc[j]
tfidfLU = {}
for i in regionsBBC:
    tfidfLU[i]=tfidf.loc[i].to_dict()


In [336]:
TFtopicLU = {
    'health': 'health',
    'racial': 'ethnicity',
    'population': 'population',
    'age': 'agemed',
    'work': 'economic',
    'commuter': 'travel',
    'housing': 'tenure'
}
TFtopicRev = dict((TFtopicLU[x],x) for x in TFtopicLU)

In [337]:
def this(lad):
    return[l for l in lads if l['name']==lad][0]

In [338]:
topics = [{1: 'population', 2: 'value', 3: 'all'}, 
          {1: 'density', 2: 'value', 3: 'all'},
          {1: 'agemed', 2: 'value', 3: 'all'},
          {1: 'care', 2: 'perc', 3: '1to19hoursWeek'},
          {1: 'care', 2: 'perc', 3: '20to49hoursWeek'},
          {1: 'care', 2: 'perc', 3: '40PlushoursWeek'},
          {1: 'children', 2: 'perc', 3: 'NoKids'},
          {1: 'children', 2: 'perc', 3: 'Kids'},
          {1: 'children', 2: 'perc', 3: 'NonDepKids'},
          {1: 'travel', 2: 'perc', 3: 'car_van'},
          {1: 'travel', 2: 'perc', 3: 'bus'},
          {1: 'travel', 2: 'perc', 3: 'home'},
          {1: 'travel', 2: 'perc', 3: 'train_metro'},
          {1: 'travel', 2: 'perc', 3: 'foot'},
          {1: 'tenure', 2: 'perc', 3: 'rented_social'},
          {1: 'tenure', 2: 'perc', 3: 'rented_private'},
          {1: 'tenure', 2: 'perc', 3: 'owned'},
          {1: 'religion', 2: 'perc', 3: 'Hindu'},
          {1: 'religion', 2: 'perc', 3: 'Buddhist'},
          {1: 'religion', 2: 'perc', 3: 'Jewish'},
          {1: 'religion', 2: 'perc', 3: 'Christian'},
          {1: 'religion', 2: 'perc', 3: 'Muslim'},
          {1: 'religion', 2: 'perc', 3: 'Noreligion'},
          {1: 'household', 2: 'perc', 3: 'OnePerson'},
          {1: 'household', 2: 'perc', 3: 'Married'},
          {1: 'household', 2: 'perc', 3: 'LoneParent'},
          {1: 'household', 2: 'perc', 3: 'Cohabiting'},
          {1: 'health', 2: 'perc', 3: 'bad'},
          {1: 'health', 2: 'perc', 3: 'good'},
          {1: 'ethnicity', 2: 'perc', 3: 'black'},
          {1: 'ethnicity', 2: 'perc', 3: 'white'},
          {1: 'ethnicity', 2: 'perc', 3: 'asian'},
          {1: 'marital', 2: 'perc', 3: 'Married'},
          {1: 'marital', 2: 'perc', 3: 'Seperated'},
          {1: 'marital', 2: 'perc', 3: 'Single'},
          {1: 'hoursworked', 2: 'perc', 3: 'Female1-15'},
          {1: 'hoursworked', 2: 'perc', 3: 'Male1-15'},
          {1: 'hoursworked', 2: 'perc', 3: 'Female49plus'},
          {1: 'hoursworked', 2: 'perc', 3: 'Male49plus'},
          {1: 'economic', 2: 'perc', 3: 'employee'},
          {1: 'economic', 2: 'perc', 3: 'student'},
          {1: 'economic', 2: 'perc', 3: 'unemployed'},
          {1: 'economic', 2: 'perc', 3: 'self-employed'}]

ladCodes = []
for lad in lads:
    ladCodes.append(lad['code'])
    
    
df = pd.DataFrame(columns=
                  ['lad', '2001',  '2011',  'change', 'topic', 'parent'])
for j in range(len(topics)):
    df1 = pd.DataFrame(index=ladCodes, columns=
                      ['lad', '2001',  '2011',  'change', 'topic', 'parent'])
    for i in range(len(lads)):
        df1['lad'].iloc[i] = lads[i]['name']
        df1['2001'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2001'][topics[j][3]]
        df1['2011'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2011'][topics[j][3]]    
        df1['change'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['change'][topics[j][3]]
        df1['topic'].iloc[i] = topics[j][1]+"_"+topics[j][3]
        df1['parent'].iloc[i] = lads[i]['parents'][0]['name']
    df = pd.concat([df,df1])


In [339]:
subjectList = ["Male31-48", "Male16-30", "Female31-48", "Female16-30", "Widowed", "65andOver", "Other", "noCare", "Sikh", "Religionnotstated", "MultiOther", "Otherreligion", "fair", "rent_free", "shared_ownership", "bicycle", "taxi", "moto", "other", "female", "male", "inactive", "mixed", "LoneNKids", "MultiStudents"]
topicList = ["age10yr"]

# Find each LAD local and national ranks and put in prioirity order
for thisLad in lads:
    
    try:
        thisLad['similar'] = [
            {'code': simiDF[simiDF['Code']==thisLad['code']]['Code.1'].iloc[0], 'name': simiDF[simiDF['Code']==thisLad['code']]['Name.1'].iloc[0]},
            {'code': simiDF[simiDF['Code']==thisLad['code']]['Code.2'].iloc[0], 'name': simiDF[simiDF['Code']==thisLad['code']]['Name.2'].iloc[0]},
            {'code': simiDF[simiDF['Code']==thisLad['code']]['Code.3'].iloc[0], 'name': simiDF[simiDF['Code']==thisLad['code']]['Name.3'].iloc[0]}
        ]
    except:
        thisLad['similar'] = [
            {'code': 'faulty pre-process', 'name': 'faulty pre-process'},
            {'code': 'faulty pre-process', 'name': 'faulty pre-process'},
            {'code': 'faulty pre-process', 'name': 'faulty pre-process'}
        ]
    
    if (areaClassDF[areaClassDF['Code']==thisLad['code']]['Subgroup code'].shape[0]>0):
        thisLad['classification'] = areaClassLU[areaClassDF[areaClassDF['Code']==thisLad['code']]['Subgroup code'].iloc[0]]
    else:
        thisLad['classification'] = "unk"
    thisLad['gss'] = {}
    thisLad['gss']['short'] = gssLookupShort[thisLad['code'][:3]]
    thisLad['gss']['long'] = gssLookup[thisLad['code'][:3]]
    
    if thisLad['parents'][0]['type'] == "cty":
        thisLad['parents'] = thisLad['parents'][1:]
    
    # Empty array will be populted with ranks for each variable
    ranks = []

    # Filter areas with same parent
    sister_lads = []
    try:
        par_name = next(i for i in thisLad['parents'] if ([j for j in i.values()][2] == 'rgn'))['name']
    except:
        par_name = next(i for i in thatLad['parents'] if ([j for j in i.values()][2] == 'ctry'))['name']
        
    for thatLad in lads:
        try:
            if  par_name == next(i for i in thatLad['parents'] if ([j for j in i.values()][2] == 'rgn'))['name']:
                sister_lads.append(thatLad)
        except:
            if par_name=='Wales':
                sister_lads.append(thatLad)
            
    # Loop through the various data variables
    for a in thisLad['data']:
        if a not in topicList:
            if a in ['population', 'density', 'agemed']:
                b = 'value'
            else:
                b = 'perc'
            # Create nested object with localised rank
            thisLad['data'][a][b+"_rank_local"] = {}    
            for c in ['2001', '2011', 'change']:
                thisLad['data'][a][b+"_rank_local"][c] = {}
                for d in thisLad['data'][a][b][c]:
                    if d not in subjectList:
                        vari = thisLad['data'][a][b][c][d]

                        # Create sorted list of values from sister areas
                        group_values = []
                        for lad in sister_lads:
                            group_values.append(lad['data'][a][b][c][d])
                            group_values = [x if (type(x) == float) | (type(x) == int) else np.nan for x in group_values]
                        group_values.sort(reverse=True)

                        # Find index of value of area of interest
                        varRank = group_values.index(vari) + 1

                        # Convert bottom half rankings into negative values
                        if varRank>len(group_values)/2:
                            varRank = varRank-len(group_values)-1

                        if thisLad['parents'][0]['name'] == "Wales":
                            natRank = [i for i in df[(df['parent']=="Wales")&(df['topic']==a+"_"+d)].sort_values(by=[c], ascending=False)['lad']].index(thisLad['name'])+1
                            if natRank > noOfLads["Wales"]/2:
                                natRank = natRank-noOfLads["Wales"]-1
                        else:
                            natRank = [i for i in df[(df['parent']!="Wales")&(df['topic']==a+"_"+d)].sort_values(by=[c], ascending=False)['lad']].index(thisLad['name'])+1
                            if natRank > noOfLads["England"]/2:
                                natRank = natRank-noOfLads["England"]-1

                        thisLad['data'][a][b+"_rank_local"][c][d] = varRank
                        thisLad['data'][a][b+"_rank"][c][d] = natRank

                        # Append ranking data to original array
                        ranks.append({
                            'label': a+'_'+b+'_'+c+'_'+d, 
                            'locRank': varRank, 
                            'natRank': natRank, 
                            'value': vari})

    # Sort in rank order
    ranks = sorted(ranks, key=lambda x: (abs(x['locRank']), -abs(x['value'])))
    thisLad["Priorities"] = ranks

In [340]:
df = pd.DataFrame(columns=
                  ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
for j in range(len(topics)):
    df1 = pd.DataFrame(index=ladCodes, columns=
                      ['lad', '2001',  '2011',  'change', 'natRank', 'localRank', 'topic', 'parent'])
    for i in range(len(lads)):
        df1['lad'].iloc[i] = lads[i]['name']
        df1['2001'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2001'][topics[j][3]]
        df1['2011'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2011'][topics[j][3]]    
        df1['change'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['change'][topics[j][3]]
        df1['natRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank"]['change'][topics[j][3]]    
        df1['localRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank_local"]['change'][topics[j][3]]    
        df1['topic'].iloc[i] = topics[j][1]+"_"+topics[j][3]
        try:
            df1['parent'].iloc[i] = next(k for k in lads[i]['parents'] if ([j for j in k.values()][2] == 'rgn'))['name']
        except:
            df1['parent'].iloc[i] = next(k for k in lads[i]['parents'] if ([j for j in k.values()][2] == 'ctry'))['name']
        
    df = pd.concat([df,df1])

for topic in topics:
    for lad in lads:
        
        # Find the areas that this area has overtaken
        v2001 = lad['data'][topic[1]][topic[2]]['2001'][topic[3]]
        v2011 = lad['data'][topic[1]][topic[2]]['2011'][topic[3]]
        if thisLad['parents'][0]['name'] == "Wales":
            dfT = df[(df['parent']=="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)]
        else:
            dfT = df[(df['parent']!="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)]

        obje = []
        for i in range(dfT.shape[0]):
            obje.append(dfT.iloc[i]['lad'])
        
        if 'overtake' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['overtake'] = {}
        lad['data'][topic[1]][topic[2]+"_rank"]['overtake'][topic[3]] = obje
        
        
        # Find areas within the region that this area has overtaken
        reg = lad['parents'][0]['name']
        dfTr = df[(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)&(df['parent']==reg)]
        objeR = []
        for i in range(dfTr.shape[0]):
            objeR.append(dfTr.iloc[i]['lad'])
        if 'overtake' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['overtake'] = {}
        lad['data'][topic[1]][topic[2]+"_rank_local"]['overtake'][topic[3]] = objeR
        
        dfReg = df[(df['topic']==topic[1]+"_"+topic[3])&(df['parent']==reg)]
        rank2001 = sorted(dfReg['2001'], reverse=True).index(v2001)+1
        if rank2001 > 168:
            rank2001 = rank2001-336-1
        if '2001' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['2001'] = {}
        lad['data'][topic[1]][topic[2]+'_rank_local']['2001'][topic[3]] = rank2001
        
        
        # Find the area immediatly above or below
        above = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]-2
        below = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]
        if below<0:
            above = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]-1
            below = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]+1
            
        name_above = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[above]['lad']
        name_below = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[below]['lad']

        area_above = {'name': name_above,
                         'value': df[(df['lad']==name_above)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}
        area_below = {'name': name_below,
                          'value': df[(df['lad']==name_below)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}

        
        if 'above_below' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'] = {}

        if topic[3] not in lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]] = {}
        
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["above"] = area_above
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["below"] = area_below
            
        # Add top and bottom three biggest movers for each subject to every area
        if thisLad['parents'][0]['name'] == "Wales":
            df_topic = df[(df['parent']=="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank'])<4)]
        else:
            df_topic = df[(df['parent']!="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank'])<4)]

        ob = {}
        for index, row in df_topic.iterrows():
            ob[int(row['natRank'])] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
        if 'top_bottom' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'] = {}
        lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'][topic[3]]=ob
        
        
        # Add top and bottom three biggest movers for each subject to every area (regional)
        df_topicReg = df[(df['topic']==topic[1]+"_"+topic[3])&(abs(df['localRank'])<4)&(df['parent']==reg)]

        ob = {}
        for index, row in df_topicReg.iterrows():
            ob[int(row['localRank'])] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
        if 'top_bottom' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['top_bottom'] = {}
        lad['data'][topic[1]][topic[2]+'_rank_local']['top_bottom'][topic[3]]=ob

In [341]:
df

,lad,2001,2011,change,natRank,localRank,topic,parent
E06000001,Hartlepool,88611.0,92028.0,3.86,-73,-6,population_all,North East
E06000002,Middlesbrough,134855.0,138412.0,2.64,-43,-4,population_all,North East
E06000003,Redcar and Cleveland,139132.0,135177.0,-2.84,-5,-2,population_all,North East
E06000004,Stockton-on-Tees,178408.0,191610.0,7.4,128,3,population_all,North East
E06000005,Darlington,97838.0,105564.0,7.9,118,2,population_all,North East
...,...,...,...,...,...,...,...,...
W06000020,Torfaen,4.93,5.83,0.9,-10,-10,economic_self-employed,Wales
W06000021,Monmouthshire,10.49,12.04,1.55,1,1,economic_self-employed,Wales
W06000022,Newport,5.5,6.56,1.06,9,9,economic_self-employed,Wales
W06000023,Powys,16.81,17.39,0.58,-4,-4,economic_self-employed,Wales


In [342]:
lads_copy = lads.copy()

In [343]:
for thisLad in lads:
    thisLad['similarC'] = thisLad['similar']

In [344]:
lads = lads_copy.copy()
# Filter out priority list by subject
for lad in lads:
    subjectList = ["Male31-48", "Male16-30", "Female31-48", "Female16-30", "Widowed", "65andOver", "Other", "noCare", "Sikh", "Religionnotstated", "MultiOther", "Otherreligion", "fair", "rent_free", "shared_ownership", "bicycle", "taxi", "moto", "other", "female", "male", "inactive", "mixed", "LoneNKids", "MultiStudents"]
    priorities = []
    priorities2011 = []
    for rank in lad['Priorities']:
        s=rank['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        if ((s[2] == "change") & (s[3] not in subjectList) & (((s[0]=="household")&(s[3]=="Married")) is False)):
            rank['2011']=[i for i in this("Manchester")['Priorities'] if i['label']==s[0]+'_'+s[1]+'_2011_'+s[3]][0]['value']
            priorities.append(rank)
        if (s[2]!="change"):
            priorities2011.append(rank)
    lad['pri'] = priorities
    lad['pri2011'] = priorities2011


# Create triple data for areas of closest proximity
geogTriples = []
for lad in lads:
    list1 = lad['bounds'][0]+lad['bounds'][1]
    for lad2 in lads:
        list2 = lad2['bounds'][0]+lad2['bounds'][1]
        listDif = [abs(list1[i]-list2[i]) for i in [0,1,2,3]]
        listDif.sort()
        if (sum(listDif[:3]) < 1) & (lad!=lad2):
            geogTriples.append([lad['name'], lad2['name'], ("near", round(sum(listDif[:3]), 2))])


sign = lambda x: math.copysign(1, x)

for thisLad in lads:
    thisLad['similar'] = thisLad['similarC']
    
    # Find nearby area of same area type
    nearSimilar = []
    for i in geogTriples:
        try:
            if ((i[0]==thisLad['name'])&(typeLookup[i[1]]==typeLookup[thisLad['name']])):
                nearSimilar.append([i[0], i[1], ('near_similar', i[2][1])])
        except:
            pass

    if len(nearSimilar)==0:
        nearSimilar = [i for i in geogTriples if i[0]==thisLad['name']]

    nearSimilar = sorted(nearSimilar, key=lambda x: x[2][1])

    
    #Find differences in data
    if (thisLad['parents'][0]['name']=='Wales'):                     
        region=[i for i in countries if i['name']=="Wales"][0]
        country=[i for i in countries if i['name']=="Wales"][0]
    else:
        region=[i for i in regions if i['name']==thisLad['parents'][0]['name']][0]
        country=[i for i in countries if i['name']==thisLad['parents'][1]['name']][0]

    def reg(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return region['data'][s[0]][s[1]][s[2]][s[3]]
    regDiff = [(i, {'val': reg(i), 'dif': i['value']-reg(i)}) for i in thisLad['pri']]
    regDiff = sorted(regDiff, reverse=True, key=lambda x: abs(x[1]['dif']))

    def cou(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return country['data'][s[0]][s[1]][s[2]][s[3]]
    couDiff = [(i, {'val': cou(i), 'dif': i['value']-cou(i)}) for i in thisLad['pri']]
    couDiff = sorted(couDiff, reverse=True, key=lambda x: abs(x[1]['dif']))
    
    try:
        nearSimilarData = {}
        nearSimilarData['name'] = nearSimilar[0][1]
        nearSimilarData['data'] = [lad for lad in lads if lad['name']==nearSimilar[0][1]][0]['data']
    except:
        print("Nearby data not found for:: ", thisLad['name'])
        nearSimilarData = []
        
    try:
        similarData = {}
        if (nearSimilar[0][1]!=thisLad['similar'][0]['name']):
            try:
                similarData['name'] = thisLad['similar'][0]['name']
                similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][0]['name']][0]['data']
            except:
                try:
                    similarData['name'] = thisLad['similar'][1]['name']
                    similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][1]['name']][0]['data']
                except:
                    similarData['name'] = thisLad['similar'][2]['name']
                    similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][2]['name']][0]['data']
        else:
            try:
                similarData['name'] = thisLad['similar'][1]['name']
                similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][1]['name']][0]['data']
            except:
                similarData['name'] = thisLad['similar'][2]['name']
                similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][2]['name']][0]['data']
    
    except:
        print("Similar data not found for:: ", thisLad['name'])
        similarData = []
    
    def simi(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return similarData['data'][s[0]][s[1]][s[2]][s[3]]
    if len(similarData)>0:
        simiDiff = [(i, {'val': simi(i), 'dif': i['value']-simi(i)}) for i in thisLad['pri']]
        simiDiff = sorted(simiDiff, reverse=True, key=lambda x: abs(x[1]['dif']))


    def near(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return nearSimilarData['data'][s[0]][s[1]][s[2]][s[3]]
    if len(nearSimilarData)>0:
        nearDiff = [(i, {'val': near(i), 'dif': i['value']-near(i)}) for i in thisLad['pri']]
        nearDiff = sorted(nearDiff, reverse=True, key=lambda x: abs(x[1]['dif']))

    nearTops = sorted([(lad['name'], lad['pri'][0]) for lad in lads if lad['name'] in [i[1] for i in nearSimilar if i[0]==thisLad['name']]], key=lambda x: abs(x[1]['locRank']))

    ageBandChange = sorted([(i, thisLad['data']['age10yr']['value']['2011'][i]-thisLad['data']['age10yr']['value']['2001'][i]) for i in thisLad['data']['age10yr']['value']['2001'].keys()], 
           reverse=True, key=lambda x: x[1])

    equalAgeChange = [lad['name'] for 
     lad in lads if (lad['parents'][0]['name'] == thisLad['parents'][0]['name']) 
     & (lad['data']['agemed']['value']['change']['all']==thisLad['data']['agemed']['value']['change']['all'])]
    len(equalAgeChange)
    
    
    # Find and refine stories
    stories = []
    stories=stories+[{**i, **{'type':['pop']}} for i in thisLad['pri'] if i['label'] == 'population_value_change_all']
    stories=stories+[{**i, **{'type':['size']}} for i in thisLad['pri'] if ((abs(i['value'])/abs(i['2011'])>0.1)&(abs(i['value']) > 1))]
    stories=stories+[{**i, **{'type':['locRank']}} for i in thisLad['pri'] if (abs(i['locRank']) < 4)&(abs(i['value']) > 1)]
    stories=stories+[{**i, **{'type':['natRank']}} for i in thisLad['pri'] if (abs(i['natRank']) < 6)&(abs(i['value']) > 1)]
    stories=stories+[{**i[0], **{'type':['couBuck']}} for i in couDiff if ((abs(i[1]['val'])>0.5) & (abs(i[0]['value'])>0.5) & (sign(i[1]['val'])!=sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['regBuck']}} for i in regDiff if ((abs(i[1]['val'])>0.5) & (abs(i[0]['value'])>0.5) & (sign(i[1]['val'])!=sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['nearBuck']}} for i in nearDiff if ((abs(i[1]['val'])>0.8) & (abs(i[0]['value'])>0.8) & (sign(i[1]['val'])!=sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['simiBuck']}} for i in simiDiff if ((abs(i[1]['val'])>0.8) & (abs(i[0]['value'])>0.8) & (sign(i[1]['val'])!=sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['couDiff']}} for i in couDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['val'])==sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['regDiff']}} for i in regDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['val'])==sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['nearDiff']}} for i in nearDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['val'])==sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['simiDiff']}} for i in simiDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['val'])==sign(i[0]['value'])))]
    seen = set()
    uniq = [item['label'] for item in stories if item['label'] in seen or seen.add(item['label'])]
    listofdoubles = [[i for i in stories if i['label'] == uniqI] for uniqI in uniq]
    multi = [{**double[0], **{'type': [item for sublist in [i['type'] for i in double] for item in sublist]}} for double in listofdoubles]
    stories = [item for item in stories if item['label'] not in [i['label'] for i in multi]]
    stories=stories+multi
    
    storiesRefined =[]
    notInc = ['density', 'age10yr', 'travel']
    for i in stories:
        if i['label'].split("_")[0] not in notInc:
            storiesRefined.append(i)
            notInc.append(i['label'].split("_")[0])

    if (thisLad['code'][0]!='W'):
        storiesRefined = sorted(storiesRefined, key=lambda x: ((topicScoresOrder[bbcRegLU[thisLad['code']]][x['label'].split('_')[0]])*(1/(1+abs(x['value'])))))
    else:
        storiesRefined = sorted(storiesRefined, key=lambda x: 1/(1+abs(x['value'])))

    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['locRank']<5)
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['natRank']<5)
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['type']=='regBuck')
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['type']=='couBuck')
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['label']=='population_value_change_all')
    storiesRefined = storiesRefined[:6]
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['label']=='agemed_value_change_all')
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['label']=='population_value_change_all')

    
    # Create a varibale 'hiRank' which holds the index of a top 5 rank at the national or reginal level to be used for an overtake story
    hiRank ={}
    hiRankNat = []
    for i in storiesRefined:
        if i['label'] not in ['population_value_change_all', 'agemed_value_change_all']:
            s=i['label'].split("_")
            if len(s)>4:
                s[3] = s[3] + "_" + s[4]
            if thisLad['data'][s[0]][s[1]+"_rank"]['2011'][s[3]]!=thisLad['data'][s[0]][s[1]+"_rank"]['2001'][s[3]]:
                hiRankNat.append(abs(thisLad['data'][s[0]][s[1]+"_rank"]['2011'][s[3]]))
            else:
                hiRankNat.append(100)
        else:
            hiRankNat.append(100)

    hiRankReg = []
    for i in storiesRefined:
        if i['label'] not in ['population_value_change_all', 'agemed_value_change_all']:
            s=i['label'].split("_")
            if len(s)>4:
                s[3] = s[3] + "_" + s[4]
            if thisLad['data'][s[0]][s[1]+"_rank_local"]['2011'][s[3]]!=thisLad['data'][s[0]][s[1]+"_rank_local"]['2001'][s[3]]:
                hiRankReg.append(abs(thisLad['data'][s[0]][s[1]+"_rank_local"]['2011'][s[3]]))
            else:
                hiRankReg.append(100)
        else:
            hiRankReg.append(100)
            
    if min(hiRankNat)<5:
        hiRank['rankIn'] = hiRankNat.index(min(hiRankNat))
        hiRank['type'] = 'nat'
    elif min(hiRankReg)<5:
        hiRank['rankIn'] = hiRankReg.index(min(hiRankReg))
        hiRank['type'] = 'reg'
    elif min(hiRankNat)<10:
        hiRank['rankIn'] = hiRankNat.index(min(hiRankNat))
        hiRank['type'] = 'nat'
    else:
        hiRank['rankIn'] = "None"
        hiRank['type'] = "None"
        
    ageBandChange= [i for i in ageBandChange if i[0]!='all']
    ageBandPos = sorted([i for i in ageBandChange if i[1]>0], reverse=True, key=lambda x: abs(x[1]))
    ageBandNeg = sorted([i for i in ageBandChange if i[1]<0], reverse=True, key=lambda x: abs(x[1]))
    

    # Add data to object
    thisLad['hiRank']=hiRank
    thisLad['stories'] = storiesRefined
    thisLad['similar'] = similarData
    thisLad['nearSimilar'] = {}
    thisLad['nearSimilar']['triples'] = nearSimilar
    thisLad['nearSimilar']['nearTops'] = nearTops
    thisLad['nearSimilar']['nearTops'] = nearSimilarData
    thisLad['differences'] = {}
    thisLad['differences']['country'] = couDiff
    thisLad['differences']['region'] = regDiff
    thisLad['differences']['near'] = nearDiff
    thisLad['data']['age10yr']['absChange'] = {}
    thisLad['data']['age10yr']['absChange']['pos'] = ageBandPos
    thisLad['data']['age10yr']['absChange']['neg'] = ageBandNeg
    thisLad['data']['agemed']['value_rank_local']['equalAgeChange]'] = len(equalAgeChange)

Similar data not found for::  Cornwall
Nearby data not found for::  Isles of Scilly
Similar data not found for::  Isles of Scilly
Similar data not found for::  Bournemouth, Christchurch and Poole
Similar data not found for::  Dorset
Similar data not found for::  Buckinghamshire
Similar data not found for::  East Suffolk
Similar data not found for::  West Suffolk
Similar data not found for::  Somerset West and Taunton


In [345]:
this("Amber Valley")['data']['density']['value_rank_local']['2011']['all']

15

In [346]:
this("Amber Valley")['data']['density']['value']['2011']['all']

4.61

In [347]:
len([l for l in lads if (l['data']['density']['value']['2011']['all']<4.61)])-307

-171

In [348]:
for rgn in regions:
    try:
        with open('/Users/theojolliffe/Documents/census-data-transformed/json/place/'+rgn['code']+'.json', 'w') as outfile:
            json.dump(rgn, outfile)
    except:
        print("Failed: ", rgn['name'])

In [349]:
for ctry in countries:
    try:
        with open('/Users/theojolliffe/Documents/census-data-transformed/json/place/'+ctry['code']+'.json', 'w') as outfile:
            json.dump(ctry, outfile)
    except:
        print("Failed: ", ctry['name'])

In [350]:
for lad in lads:
    try:
        with open('/Users/theojolliffe/Documents/census-data-transformed/json/place/'+lad['code']+'.json', 'w') as outfile:
            json.dump(lad, outfile)
    except:
        print("Failed: ", lad['name'])


In [353]:
df.to_csv('laddata.csv')

In [354]:
!git add .
!git commit -m "Add files"
!git push -u origin main

[main e0defab4] Add files
 331 files changed, 14991 insertions(+), 488 deletions(-)
 rewrite json/place/E06000001.json (95%)
 rewrite json/place/E06000002.json (95%)
 rewrite json/place/E06000003.json (95%)
 rewrite json/place/E06000004.json (92%)
 rewrite json/place/E06000005.json (86%)
 rewrite json/place/E06000006.json (95%)
 rewrite json/place/E06000007.json (95%)
 rewrite json/place/E06000008.json (93%)
 rewrite json/place/E06000009.json (95%)
 rewrite json/place/E06000010.json (95%)
 rewrite json/place/E06000011.json (94%)
 rewrite json/place/E06000012.json (94%)
 rewrite json/place/E06000013.json (94%)
 rewrite json/place/E06000014.json (95%)
 rewrite json/place/E06000015.json (94%)
 rewrite json/place/E06000016.json (94%)
 rewrite json/place/E06000017.json (94%)
 rewrite json/place/E06000018.json (93%)
 rewrite json/place/E06000019.json (94%)
 rewrite json/place/E06000020.json (95%)
 rewrite json/place/E06000021.json (95%)
 rewrite json/place/E06000022.json (94%)
 rewrite json/

In [240]:
type(lads[2])

dict

In [352]:
df

,lad,2001,2011,change,natRank,localRank,topic,parent
E06000001,Hartlepool,88611.0,92028.0,3.86,-73,-6,population_all,North East
E06000002,Middlesbrough,134855.0,138412.0,2.64,-43,-4,population_all,North East
E06000003,Redcar and Cleveland,139132.0,135177.0,-2.84,-5,-2,population_all,North East
E06000004,Stockton-on-Tees,178408.0,191610.0,7.4,128,3,population_all,North East
E06000005,Darlington,97838.0,105564.0,7.9,118,2,population_all,North East
...,...,...,...,...,...,...,...,...
W06000020,Torfaen,4.93,5.83,0.9,-10,-10,economic_self-employed,Wales
W06000021,Monmouthshire,10.49,12.04,1.55,1,1,economic_self-employed,Wales
W06000022,Newport,5.5,6.56,1.06,9,9,economic_self-employed,Wales
W06000023,Powys,16.81,17.39,0.58,-4,-4,economic_self-employed,Wales
